# Importing supporting libraries

In [2]:
%%time
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

import pandas as pd
import pprint

CPU times: total: 766 ms
Wall time: 10.1 s


# Connecting Local SQL Server

In [3]:
engine = create_engine(
        "mssql+pyodbc://INBLRWCXX21J3\\SQLEXPRESS/Matrix?"
        "driver=ODBC+Driver+17+for+SQL+Server"
        "&autocommit=true"
        "&trusted_connection=yes"
        "&echo=False"
        "&fast_executemany=True"
    )

# Check if the connection is successful
if engine.connect():
    print("Connection successful!")
else:
    print("Connection failed.")

Connection successful!


## Get database schema

In [4]:
def get_schema(table_list: list):
       
    # Query to get table and column information
    table_name = ', '.join(f"'{name}'" for name in table_list)

    schema_query = f"""
    SELECT 
        t.name AS table_name,
        c.name AS column_name,
        ty.name AS data_type
    FROM sys.tables t
    INNER JOIN sys.columns c ON t.object_id = c.object_id
    INNER JOIN sys.types ty ON c.user_type_id = ty.user_type_id
    where t.name in ({table_name})
    ORDER BY t.name, c.column_id
    """
    
    with engine.connect() as conn:
        result = conn.execute(text(schema_query))  # Execute query
        schema_info = result.fetchall()  # Fetch all rows
    
    # Format schema information
    schema_text = "Database Schema:\n"
    current_table = ""
    
    for table, column, data_type in schema_info:
        if table != current_table:
            schema_text += f"\nTable: {table}\n"
            current_table = table
        schema_text += f"- {column} ({data_type})\n"
    
    conn.close()
    
    return schema_text

schema_text = get_schema(table_list = ['Employee'])
print(schema_text)

Database Schema:

Table: Employee
- Employee_Id (varchar)
- Employee_Name (varchar)
- Group_Name (varchar)
- Manager_Id (varchar)
- Manager_Name (varchar)



# Connecting with GROQ API

In [12]:
llm = ChatGroq(
    groq_api_key = os.getenv('groq_api_key'),
    model_name="mixtral-8x7b-32768" , # You can also use "llama2-70b-4096"
)

# Create prompt template

In [6]:
user_query = "How many employees are there in the organization?"

In [7]:
prompt = PromptTemplate.from_template(f"""
You are a SQL expert. Based on the following database schema and natural language query,
generate a SQL query that answers the question.

{schema_text}

Natural Language Query: {user_query}

Generate only the SQL query without any explanation or additional text.
The query should be compatible with Microsoft SQL Server.
""")

print(prompt.template)



You are a SQL expert. Based on the following database schema and natural language query,
generate a SQL query that answers the question.

Database Schema:

Table: Employee
- Employee_Id (varchar)
- Employee_Name (varchar)
- Group_Name (varchar)
- Manager_Id (varchar)
- Manager_Name (varchar)


Natural Language Query: How many employees are there in the organization?

Generate only the SQL query without any explanation or additional text.
The query should be compatible with Microsoft SQL Server.



In [10]:
def process_natural_language_query(nl_query):
    try:
        # Get database schema
        schema = get_schema(table_list = ['Employee'])
        
        # Generate SQL query using the new chain syntax
        sql_query = chain.invoke({
            "schema": schema,
            "query": nl_query
        })
        
        print("Generated SQL Query:")
        print(sql_query)
        
        # Execute the query
        results = pd.read_sql(sql_query,con=engine)
        
        return {
            "sql_query": sql_query,
            "results": results
        }
        
    except Exception as e:
        import traceback
        print(f"Full error traceback:\n{traceback.format_exc()}")
        return f"Error processing query: {str(e)}"

In [11]:
nl_query = "How many employees are there in the organization?"
    
# Process the query
result = process_natural_language_query(nl_query)
print("\nResults:")
print(result)

Full error traceback:
Traceback (most recent call last):
  File "c:\Users\a410155\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpx\_transports\default.py", line 72, in map_httpcore_exceptions
    yield
  File "c:\Users\a410155\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpx\_transports\default.py", line 236, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\a410155\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_sync\connection_pool.py", line 216, in handle_request
    raise exc from None
  File "c:\Users\a410155\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_sync\connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\a410155\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_sync\connection.py", line 99, in handle_request
    raise exc


In [13]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Create the chain using the new syntax
chain = ({"schema": RunnablePassthrough(),"query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

sql_query = chain.invoke({
            "schema": schema_text,
            "query": user_query
        })

APIConnectionError: Connection error.

In [ ]:
import requests
groq_api_key = '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'
api_key = os.getenv("groq_api_key")
headers = {"Authorization": f"Bearer {api_key}"}
response = requests.get("https://api.groq.com/v1/models", headers=headers, verify=False)  # Try verify=True first
print(response.json())  # Should return available models

c:\Users\a410155\OneDrive - Volvo Group\Volvo\Studies\Gen AI\Assignment Solutions\Chatbot SQL\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.groq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'error': {'message': 'Unknown request URL: GET /v1/models. Please check the URL for typos, or see the docs at https://console.groq.com/docs/', 'type': 'invalid_request_error', 'code': 'unknown_url'}}


In [16]:
pwd

'c:\\Users\\a410155\\OneDrive - Volvo Group\\Volvo\\Studies\\Gen AI\\Assignment Solutions\\Chatbot SQL'